In [822]:
import requests
import json
import io
import numpy as np
import re

## Insert data

In [948]:
ts1 = '{"10": {"ts": {"times": [1, 2, 3], "values": [2, 4, 9]}}}'
ts2 = '{"12": {"ts": {"times": [1, 2, 3], "values": [1, 5, 10]}}}'

In [949]:
r = requests.post('http://127.0.0.1:5000/insert', json = ts1)
r = requests.post('http://127.0.0.1:5000/insert', json = ts2)

## Request data

In [923]:
r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))

In [924]:
# r.json()

In [925]:
r = requests.get('http://127.0.0.1:5000/select?fields=ts', auth=('user', 'pass'))
# r.json()

## Insert meta

In [950]:
meta = {'10': {'order': 1, 'blarg': 1}}
meta2 = {'12': {'order': 2, 'blarg': 2}}

#  db.upsert_meta('one', {'order': 1, 'blarg': 1})
print(meta)
s_meta = io.StringIO()
json.dump(meta, fp=s_meta)
s_meta.getvalue()

s_meta2 = io.StringIO()
json.dump(meta2, fp=s_meta2)
s_meta2.getvalue()

{'10': {'order': 1, 'blarg': 1}}


'{"12": {"order": 2, "blarg": 2}}'

In [951]:
r = requests.post('http://127.0.0.1:5000/upsert_meta', json = s_meta.getvalue())
r = requests.post('http://127.0.0.1:5000/upsert_meta', json = s_meta2.getvalue())

In [798]:
# r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))
# r.json()

## Sort by order

In [523]:
r = requests.get('http://127.0.0.1:5000/select?additional=sort_by,%2Border', auth=('user', 'pass'))
r.json()

{'10': {'blarg': 1, 'order': 1, 'pk': '10'},
 '12': {'blarg': 2, 'order': 2, 'pk': '12'}}

In [399]:
r = requests.get('http://127.0.0.1:5000/select?additional=sort_by,%2Border,limit,1', auth=('user', 'pass'))
r.json()

{'10': {'blarg': 1, 'order': 1, 'pk': '10'}}

In [53]:
jsonfile = r.json()
jsonfile

{'10': {'ts': {'times': [1, 2, 3], 'values': [4, 5, 6]}}}

In [194]:
s = io.StringIO()
json.dump(jsonfile, fp=s)
s.getvalue()

'{"10": {"ts": {"values": [4, 5, 6], "times": [1, 2, 3]}}}'

In [260]:
other_value = '{"10": {"ts": {"times": [1, 2, 3], "values": [2, 4, 9]}}}'

In [267]:
r = requests.post('http://127.0.0.1:5000/insert', json = s.getvalue())

In [81]:
s.getvalue()

'{"10": {"ts": {"values": [4, 5, 6], "times": [1, 2, 3]}}}'

In [86]:
data = json.loads(s.getvalue())

In [643]:
r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))
r.json()

{'10': {'pk': '10'}, '12': {'pk': '12'}}

## Filter by meta

In [689]:
r = requests.get('http://127.0.0.1:5000/select?md=blarg,<2', auth=('user', 'pass'))
# {'blarg': {'<=': 2}}
r.json()

{}

## Select augmented

In [978]:
augmented = {'md': {"blarg": {'>=': 1}},'proc':'stats','target':['damean','dastd'], 'additional':{"sort_by": "+order", "limit": 2}}
augemented_json = io.StringIO()
json.dump(augmented, fp=augemented_json)
augmented_data = augemented_json.getvalue()
augmented_data

'{"target": ["damean", "dastd"], "proc": "stats", "md": {"blarg": {">=": 1}}, "additional": {"limit": 2, "sort_by": "+order"}}'

In [979]:
r = requests.post('http://127.0.0.1:5000/augmented_select', json = augmented_data)
# print (r.text)
r.json()

{'10': {'damean': 5.0, 'dastd': 2.943920288775949},
 '12': {'damean': 5.333333333333333, 'dastd': 3.6817870057290873}}